In [ ]:
from dataclasses import asdict
from dp_virtual_projection_population_only import ExperimentConfig as VProjConfig, train as vp_train
from local_dpsgd_experiment import ExperimentConfig as LDConfig, train as ld_train
from experiment_utils import clear_memory


In [ ]:
# Virtual projection experiment
vp_cfg = VProjConfig(
    num_epochs=1,
    batch_size=512,
    lr=0.05,
    outer_momentum=0.9,
    inner_momentum=0.10,
    noise_mult=1.0,
    delta=1e-5,
    c_start=4.0,
    c_end=2.0,
    self_aug_factor=1,
    schedule_milestones=[12, 18],
    schedule_gamma=0.1,
    max_momentum_size=10000,
)
print(asdict(vp_cfg))
vp_results = vp_train(vp_cfg, 'outputs')
clear_memory()
vp_results


In [ ]:
# Local DP-SGD experiment
ld_cfg = LDConfig(
    EXPERIMENT='local_pub',
    PRIV_BATCH=512,
    NUM_EPOCHS=1,
    LR_OUTER=0.05,
    MOMENTUM_OUTER=0.9,
)
print(asdict(ld_cfg))
ld_results = ld_train(ld_cfg, 'outputs')
clear_memory()
ld_results
